# 🛠️ The Data Pipeline

Welcome to the engine room! This notebook is responsible for taking the raw ride data from Divvy and transforming it into a clean, structured format ready for analysis. We'll be engineering features that help us distinguish between casual riders and members, ultimately producing our core datasets: the `dim_stations` and `fact_trips` tables.

### 1. Essential Imports
First, we bring in our standard toolkit for data manipulation and file path management.

In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

### 2. Setting Up Paths
Let's define where our raw data lives and where we want to save our processed results. We also set a threshold for station stability to ensure our analysis is robust.

In [ ]:
RAW_DIR = Path("../data/raw")
PROCESSED_DIR = Path("../data/processed")
MIN_TRIPS_FOR_STABILITY = 50

### 3. Loading the Raw Data
We scan our raw data directory for CSV files and pull them all together. To keep things efficient, we're only selecting the columns we actually need for our behavioral modeling.

In [6]:
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

all_files = list(RAW_DIR.glob("*.csv"))
if not all_files:
    raise FileNotFoundError(f"No CSVs found in {RAW_DIR}.")

keep_cols = [
    'ride_id', 'started_at', 'ended_at', 'member_casual',
    'start_station_name', 'start_lat', 'start_lng', 'end_lat', 'end_lng'
]

print(f"Processing {len(all_files)} files...")
df = pd.concat(
    (pd.read_csv(f, usecols=lambda x: x.lower() in keep_cols) for f in all_files),
    ignore_index=True
)
df.columns = df.columns.str.lower()
print(f"Loaded {len(df):,} rows.")

Processing 13 files...
Loaded 5,731,366 rows.


### 4. Cleaning the Records
Time to tidy up! We convert our timestamps, drop any incomplete records, and calculate the duration of each ride. We also filter out any rides that are impossibly short (less than a minute) or excessively long (over a day).

In [7]:
df['started_at'] = pd.to_datetime(df['started_at'], errors='coerce')
df['ended_at'] = pd.to_datetime(df['ended_at'], errors='coerce')
df = df.dropna(subset=['started_at', 'start_station_name'])

df['ride_length'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60
df = df[(df['ride_length'] > 1) & (df['ride_length'] < 1440)]
print(f"After cleaning: {len(df):,} rows.")

After cleaning: 4,451,938 rows.


### 5. Engineering Behavioral Features
To better understand *how* people use these bikes, we're adding some context. We'll identify the hour of the day, whether it's a weekday, and if the trip looks like a classic work commute.

In [8]:
df['hour'] = df['started_at'].dt.hour
df['is_weekday'] = df['started_at'].dt.dayofweek < 5
is_rush = df['hour'].isin([7, 8, 9, 10, 16, 17, 18, 19])
df['is_commute'] = is_rush & df['is_weekday'] & (df['ride_length'] <= 30)
df.head()

,ride_id,started_at,ended_at,start_station_name,start_lat,start_lng,end_lat,end_lng,member_casual,ride_length,hour,is_weekday,is_commute
0,6C960DEB4F78854E,2024-12-31 01:38:35.018,2024-12-31 01:48:45.775,Halsted St & Roscoe St,41.943632,-87.649083,41.973348,-87.667855,member,10.179283,1,True,False
1,C0913EEB2834E7A2,2024-12-21 18:41:26.478,2024-12-21 18:47:33.871,Clark St & Wellington Ave,41.936497,-87.647539,41.943632,-87.649083,member,6.123217,18,False,False
2,848A37DD4723078A,2024-12-21 11:41:01.664,2024-12-21 11:52:45.094,Sheridan Rd & Montrose Ave,41.961670,-87.654640,41.937582,-87.644098,member,11.723833,11,False,False
3,3FA09C762ECB48BD,2024-12-26 13:07:27.526,2024-12-26 13:10:54.130,Aberdeen St & Jackson Blvd,41.877726,-87.654787,41.883602,-87.648627,member,3.443400,13,True,False
4,E60317ADD1A87488,2024-12-13 15:17:55.063,2024-12-13 15:27:32.583,Paulina St & Flournoy St,41.873061,-87.669135,41.866624,-87.694521,member,9.625333,15,True,False


### 6. Exporting Station Metadata
After cleaning, we consolidate our station data. We calculate the mean coordinates for each start station and save this to `dim_stations.csv`. This will be our primary spatial reference.

In [9]:
station_dim = df.groupby('start_station_name').agg({
    'start_lat': 'mean',
    'start_lng': 'mean'
}).reset_index()
station_dim.to_csv(PROCESSED_DIR / "dim_stations.csv", index=False)
print(f"dim_stations.csv saved ({len(station_dim)} stations).")

dim_stations.csv saved (1866 stations).


### 7. Saving the Trip Master File
We finalize our pipeline by exporting the detailed trip records to `fact_trips.csv`. This file includes all our engineered features and is ready for depth behavioral analysis.

In [10]:
fact_cols = [
    'ride_id', 'start_station_name', 'started_at', 'member_casual',
    'is_commute', 'ride_length', 'start_lat', 'start_lng', 'end_lat', 'end_lng'
]
df[fact_cols].to_csv(PROCESSED_DIR / "fact_trips.csv", index=False)

print(f"\u2705 SUCCESS: Pipeline complete. Fact table contains {len(df):,} rows with coordinates.")

✅ SUCCESS: Pipeline complete. Fact table contains 4,451,938 rows with coordinates.
